In [1]:
import os
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Этап 1. Загрузка текстов из файлов

In [2]:
TEXT_FILES_DIR = "garant"

In [3]:
def load_texts_from_folder(folder_path):
    texts = []
    filenames = []
    for file in tqdm(sorted(os.listdir(folder_path)), desc="Загрузка текстов"):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
                texts.append(text)
                filenames.append(file)
    return texts, filenames

In [4]:
texts, filenames = load_texts_from_folder(TEXT_FILES_DIR)

Загрузка текстов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1628/1628 [00:02<00:00, 643.29it/s]


## Этап 2. Создание эмбеддингов

In [5]:
MODEL_NAME = "DeepPavlov/rubert-base-cased"
model = SentenceTransformer(MODEL_NAME)

No sentence-transformers model found with name DeepPavlov/rubert-base-cased. Creating a new one with mean pooling.
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model

In [6]:
text_embeddings = model.encode(texts, batch_size=16, convert_to_numpy=True, show_progress_bar=True)
print(f"\nРазмерность эмбеддингов: {text_embeddings.shape}")

Batches:   0%|          | 0/102 [00:00<?, ?it/s]


Размерность эмбеддингов: (1628, 768)


## Этап 3: Создание FAISS-индекса

FAISS используется для быстрого поиска среди эмбеддингов

In [7]:
norms = np.linalg.norm(text_embeddings, axis=1, keepdims=True)
text_embeddings_normalized = text_embeddings / norms
dimension = text_embeddings.shape[1]

Создаём FAISS-индекс

In [8]:
index = faiss.IndexFlatL2(dimension)
index.add(text_embeddings_normalized.astype('float32'))

Сохраняем индекс для последующего использования

In [9]:
faiss.write_index(index, "results_DeepPavlov_Default/text_index.faiss")
np.save("results_DeepPavlov_Default/filenames.npy", np.array(filenames))  # сохраняем список имён файлов

## Этап 4: Функция поиска похожих текстов

In [10]:
def find_similar_texts(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True) # преобразуем запрос в эмбеддинг
    distances, indices = index.search(query_embedding, top_k) # выполняем поиск в FAISS

    print("Наиболее похожие тексты:")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {filenames[idx]} (дистанция: {distances[0][i]:.4f})")
        print(texts[idx][:300] + "...")  # выводим первые 300 символов текста
        print("-" * 80)

In [11]:
query_text = "Международный день инвалидов в Хабаровске"
find_similar_texts(query_text, top_k=20)

Наиболее похожие тексты:
1. garant_1660.txt (дистанция: 62.8300)
﻿
Закон Санкт-Петербурга от 18 октября 2004 г. N 492-72
"О внесении изменения в Закон Санкт-Петербурга
"О налоге на игорный бизнес в Санкт-Петербурге"
(Принят Законодательным Собранием Санкт-Петербурга
6 октября 2004 года)

Статья 1
Внести в Закон Санкт-Петербурга от 22 октября 2003 года N 623-94 "О...
--------------------------------------------------------------------------------
2. garant_0763.txt (дистанция: 62.9080)
﻿
Постановление Правительства Ханты-Мансийского АО - Югры от 5 ноября 2004 г. N 437-п
"Об установлении величины прожиточного минимума в Ханты-Мансийском автономном округе - Югре 
за III квартал 2004 года"


В соответствии с Федеральным законом от 24.10.97 N 134-ФЗ "О прожиточном минимуме в Российско...
--------------------------------------------------------------------------------
3. garant_1261.txt (дистанция: 63.0506)
﻿
Постановление Мэра г. Хабаровска
от 11 ноября 2004 г. N 1650
"О проведении Междунар